In [2]:
# Imports - English comments
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
from astroquery.jplhorizons import Horizons
# other imports as needed...
from astropy.time import Time
from astropy.coordinates import SkyCoord, CartesianRepresentation, ICRS, HeliocentricTrueEcliptic
import astropy.units as u

import time
from requests.exceptions import HTTPError


In [3]:
# fetch or load the data, e.g. df from JPL Horizons
# df = fetch_vectors_horizons('Voyager 1', '2012-01-01', '2025-12-01', n_points=150)
# For demo, use your existing df variables
def _call_horizons_with_retries(name, epochs_arg, max_retries=4, pause_base=1.0):
    """
    Helper: call Horizons(..., epochs=epochs_arg) with retries and exponential backoff.
    epochs_arg may be either:
      - a numpy array/list of JDs (short lists are OK), or
      - a dict {'start': 'YYYY-MM-DD', 'stop': 'YYYY-MM-DD', 'step': 'Nd'} which avoids long TLIST.
    Returns pandas DataFrame (or raises if all retries fail).
    """
    attempt = 0
    while attempt < max_retries:
        try:
            obj = Horizons(id=name, location='@sun', epochs=epochs_arg)
            table = obj.vectors()
            return table.to_pandas()
        except HTTPError as e:
            # If server returned 502/5xx, retry with backoff
            attempt += 1
            wait = pause_base * (2 ** (attempt - 1))
            print(f"HTTPError on attempt {attempt}/{max_retries}: {e}. Retrying after {wait:.1f}s...")
            time.sleep(wait)
        except Exception as e:
            # Other unexpected exceptions: raise immediately
            print("Unexpected error when querying Horizons:", type(e), e)
            raise
    # if we exit loop, all retries failed
    raise RuntimeError(f"Horizons query failed after {max_retries} attempts.")

def fetch_vectors_horizons(name, start_iso, stop_iso, n_points=200, prefer_range_threshold=250):
    """
    Robust fetch from JPL Horizons:
    - If n_points is small (< prefer_range_threshold), request explicit JD list.
    - If n_points is large, request a range (start/stop/step) to avoid long TLIST URIs.
    - Returns concatenated pandas DataFrame with x,y,z columns (AU).
    """
    t0 = Time(start_iso, format='iso', scale='utc')
    t1 = Time(stop_iso, format='iso', scale='utc')

    total_days = (t1 - t0).to(u.day).value
    if n_points <= 1:
        jd_grid = np.array([t0.jd])
    else:
        jd_grid = np.linspace(t0.jd, t1.jd, n_points)

    # If many points, prefer asking Horizons using a step-range (smaller URL)
    if n_points > prefer_range_threshold:
        # compute integer day step (at least 1 day)
        step_days = max(1, int(round(total_days / max(1, n_points - 1))))
        epochs_arg = {'start': start_iso, 'stop': stop_iso, 'step': f'{step_days}d'}
        print(f"Using range request to Horizons with step = {step_days} day(s) to avoid long URL.")
        df = _call_horizons_with_retries(name, epochs_arg)
        # Note: number of returned samples may differ slightly from n_points
        return df
    else:
        # For modest sized requests, use chunking to be safe (avoid giant single TLIST)
        max_chunk = 120  # keep each TLIST under ~120 entries
        dfs = []
        for i in range(0, len(jd_grid), max_chunk):
            chunk = jd_grid[i:i+max_chunk]
            print(f"Querying Horizons for {name}: chunk {i//max_chunk + 1}, {len(chunk)} epochs...")
            df_chunk = _call_horizons_with_retries(name, chunk)
            dfs.append(df_chunk)
        # concatenate and drop possible duplicate header rows
        df_all = pd.concat(dfs, ignore_index=True)
        # Some Horizons responses include overlapping rows; optionally drop exact-duplicate epochs
        if 'datetime_jd' in df_all.columns:
            df_all = df_all.drop_duplicates(subset=['datetime_jd'])
        return df_all

In [33]:
df_v1 = fetch_vectors_horizons('Voyager 1', '1982-01-01', '2027-12-01', 3000)
# inspect
df_v1.head()


Using range request to Horizons with step = 6 day(s) to avoid long URL.


,targetname,datetime_jd,datetime_str,x,y,z,vx,vy,vz,lighttime,range,range_rate
0,Voyager 1 (spacecraft) (-31),2444970.5,A.D. 1982-Jan-01 00:00:00.0000,-10.787118,-4.310946,3.243051,-0.002620,-0.009393,0.006796,0.069658,12.060827,0.007528
1,Voyager 1 (spacecraft) (-31),2444976.5,A.D. 1982-Jan-07 00:00:00.0000,-10.802804,-4.367292,3.283815,-0.002609,-0.009389,0.006792,0.069919,12.106084,0.007558
2,Voyager 1 (spacecraft) (-31),2444982.5,A.D. 1982-Jan-13 00:00:00.0000,-10.818425,-4.423611,3.324560,-0.002598,-0.009384,0.006789,0.070181,12.151517,0.007587
3,Voyager 1 (spacecraft) (-31),2444988.5,A.D. 1982-Jan-19 00:00:00.0000,-10.833981,-4.479904,3.365284,-0.002587,-0.009380,0.006786,0.070445,12.197124,0.007616
4,Voyager 1 (spacecraft) (-31),2444994.5,A.D. 1982-Jan-25 00:00:00.0000,-10.849473,-4.536170,3.405988,-0.002577,-0.009376,0.006782,0.070709,12.242903,0.007644


In [7]:
# use the earlier plot_3d_trajectories code or quickly:
fig = go.Figure()
fig.add_trace(go.Scatter3d(x=df_v1['x'], y=df_v1['y'], z=df_v1['z'], mode='lines+markers', name='V1'))
fig.add_trace(go.Scatter3d(x=[0], y=[0], z=[0], mode='markers', marker=dict(size=6), name='Sun'))
fig.update_layout(scene=dict(aspectmode='auto'))
fig.show()


In [26]:
# Replace this function in your notebook. Comments in English.
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio

# small helper to extract JD array from typical Horizons DataFrame outputs
def _get_jd_array_from_df(df):
    """
    Try common column names to extract JD values from the dataframe.
    Returns a numpy array of floats (JD). If none found, returns None.
    """
    # common names returned by astroquery Horizons.vectors() include 'datetime_jd'
    if 'datetime_jd' in df.columns:
        return df['datetime_jd'].astype(float).values
    # some variants: 'datetime_str' (ISO strings), 'datetime' etc.
    if 'datetime_str' in df.columns:
        try:
            from astropy.time import Time
            return Time(df['datetime_str'].values, format='iso').jd
        except Exception:
            pass
    if 'datetime' in df.columns:
        try:
            from astropy.time import Time
            return Time(df['datetime'].values, format='iso').jd
        except Exception:
            pass
    # try column names that sometimes appear
    for col in ['jd', 'JDTDB', 'JD', 'time_jd', 'epoch']:
        if col in df.columns:
            try:
                return df[col].astype(float).values
            except Exception:
                pass
    # nothing found
    return None

def plot_3d_axes_at_origin(name_list, df_list, n_ticks=3, pad_factor=1.05, axis_color='black', axis_width=3, tick_font_size=10, n_sample_for_markers=None):
    """
    Plot spacecraft trajectories as lines, draw custom X/Y/Z axes that pass through origin,
    remove the default axis planes, and add tick labels along the custom axes.

    Added behavior: color per-point markers by absolute epoch (JD). That means points
    at the same physical time (same JD) across different spacecraft will have the same color.

    - n_sample_for_markers: if not None, will subsample each df's points for markers
      (int number of markers per spacecraft). If None, uses all sample points.
    """
    # Collect global bounds
    all_x = np.hstack([df['x'].values for df in df_list])
    all_y = np.hstack([df['y'].values for df in df_list])
    all_z = np.hstack([df['z'].values for df in df_list])
    max_val = max(np.nanmax(np.abs(all_x)), np.nanmax(np.abs(all_y)), np.nanmax(np.abs(all_z)))
    if max_val == 0 or np.isnan(max_val):
        max_val = 1.0
    half_range = float(max_val) * float(pad_factor)
    axis_range = [-half_range, half_range]

    fig = go.Figure()

    # add spacecraft trajectory lines (unchanged)
    for name, df in zip(name_list, df_list):
        fig.add_trace(go.Scatter3d(
            x=df['x'].values, y=df['y'].values, z=df['z'].values,
            mode='lines',
            line=dict(width=2),
            name=name,
            hoverinfo='name'
        ))

    # --- NEW: prepare time-coloring for markers ---
    # extract JD arrays per dataframe (try robustly)
    jd_list = []
    for df in df_list:
        jd = _get_jd_array_from_df(df)
        jd_list.append(jd)

    # Determine global cmin/cmax for color scaling from JD values that exist
    jd_present = [j for j in jd_list if j is not None and len(j) > 0]
    if len(jd_present) > 0:
        global_jd_min = float(min(j.min() for j in jd_present))
        global_jd_max = float(max(j.max() for j in jd_present))
    else:
        # fallback to index-based normalized time if no JD found
        global_jd_min = 0.0
        global_jd_max = 1.0

    # Attempt to build human-readable tick labels for the colorbar (if astropy available)
    colorbar_tickvals = None
    colorbar_ticktext = None
    try:
        if len(jd_present) > 0:
            from astropy.time import Time
            tickvals = np.linspace(global_jd_min, global_jd_max, 5)
            ticktext = [Time(tv, format='jd').iso.split()[0] for tv in tickvals]  # YYYY-MM-DD
            colorbar_tickvals = tickvals
            colorbar_ticktext = ticktext
    except Exception:
        colorbar_tickvals = None
        colorbar_ticktext = None

    # Add per-spacecraft marker traces colored by JD (if available)
    # We'll show the colorbar only for the first marker trace that has JD info (avoid duplicate bars).
    colorbar_shown = False
    for name, df, jd in zip(name_list, df_list, jd_list):
        # decide which sample indices to use for markers (subsample for performance if requested)
        if n_sample_for_markers is None:
            idx = np.arange(len(df))
        else:
            # choose roughly evenly spaced indices
            m = min(len(df), max(1, int(n_sample_for_markers)))
            if m >= len(df):
                idx = np.arange(len(df))
            else:
                idx = np.linspace(0, len(df) - 1, m, dtype=int)

        xs = df['x'].values[idx]
        ys = df['y'].values[idx]
        zs = df['z'].values[idx]

        if jd is not None:
            cj = jd[idx]
            # show colorbar only once
            show_cb = (not colorbar_shown)
            marker_dict = dict(
                size=1,
                color=cj,
                colorscale='Viridis',
                cmin=global_jd_min,
                cmax=global_jd_max,
                opacity=0.2,
                colorbar=dict(
                    title='Epoch (UTC)',
                    # Place the colorbar further right and slightly shorter to avoid overlapping the legend.
                    # Plotly uses paper coordinates: x in [0..1] is inside the plot box; >1 moves into the right margin.
                    x=1,
                    y=0.3,
                    yanchor='middle',
                    len=0.5,
                    thickness=24,
                    xpad=6,
                    ypad=6,
                    tickmode='array' if colorbar_tickvals is not None else 'auto',
                    tickvals=list(colorbar_tickvals) if colorbar_tickvals is not None else None,
                    ticktext=list(colorbar_ticktext) if colorbar_ticktext is not None else None,
                ) if show_cb else None
            )
            colorbar_shown = colorbar_shown or show_cb
            # create hover text as short date if possible (avoid creating huge strings for many points)
            hover_text = None
            try:
                from astropy.time import Time
                iso_dates = Time(cj, format='jd').iso
                # keep only date part YYYY-MM-DD to keep hover light
                hover_text = [s.split()[0] for s in iso_dates]
            except Exception:
                hover_text = None

            fig.add_trace(go.Scatter3d(
                x=xs, y=ys, z=zs,
                mode='markers',
                marker=marker_dict,
                name=f"{name} (epoch-colored)",
                text=hover_text,
                hoverinfo='text+name' if hover_text is not None else 'name',
                showlegend=False
            ))
        else:
            # no JD available: fall back to index-based normalized color
            idx_norm = np.linspace(0.0, 1.0, len(idx))
            marker_dict = dict(size=1, color=idx_norm, colorscale='Viridis', cmin=0.0, cmax=1.0, opacity=0.2)
            fig.add_trace(go.Scatter3d(
                x=xs, y=ys, z=zs,
                mode='markers',
                marker=marker_dict,
                name=f"{name} (index-colored)",
                hoverinfo='name',
                showlegend=False
            ))

    # add Sun marker at origin (unchanged)
    fig.add_trace(go.Scatter3d(
        x=[0.0], y=[0.0], z=[0.0],
        mode='markers+text',
        marker=dict(size=2),
        text=['Sun'],
        textposition='top center',
        name='Sun (0,0,0)'
    ))

    # Draw custom axes as lines passing through origin (unchanged)
    # X axis
    fig.add_trace(go.Scatter3d(
        x=[axis_range[0], axis_range[1]],
        y=[0.0, 0.0],
        z=[0.0, 0.0],
        mode='lines',
        line=dict(color=axis_color, width=axis_width),
        name='X axis',
        hoverinfo='none',
        showlegend=False
    ))
    # Y axis
    fig.add_trace(go.Scatter3d(
        x=[0.0, 0.0],
        y=[axis_range[0], axis_range[1]],
        z=[0.0, 0.0],
        mode='lines',
        line=dict(color=axis_color, width=axis_width),
        name='Y axis',
        hoverinfo='none',
        showlegend=False
    ))
    # Z axis
    fig.add_trace(go.Scatter3d(
        x=[0.0, 0.0],
        y=[0.0, 0.0],
        z=[axis_range[0], axis_range[1]],
        mode='lines',
        line=dict(color=axis_color, width=axis_width),
        name='Z axis',
        hoverinfo='none',
        showlegend=False
    ))

    # Create tick positions (symmetric around 0) - unchanged
    pos_ticks = np.linspace(0, half_range, n_ticks+1)  # includes 0 and half_range
    neg_ticks = -pos_ticks[::-1]  # negative side including -half_range and 0
    tick_positions = np.concatenate([neg_ticks[:-1], pos_ticks])  # avoid repeating 0
    tick_labels = [f"{t:.1f}" for t in tick_positions]

    # tick labels along axes (unchanged)
    fig.add_trace(go.Scatter3d(
        x=tick_positions, y=[0.0 + half_range*0.01]*len(tick_positions), z=[0.0 + half_range*0.01]*len(tick_positions),
        mode='text',
        text=tick_labels,
        textfont=dict(size=tick_font_size),
        hoverinfo='none',
        showlegend=False
    ))
    fig.add_trace(go.Scatter3d(
        x=[0.0 + half_range*0.01]*len(tick_positions), y=tick_positions, z=[0.0 + half_range*0.01]*len(tick_positions),
        mode='text',
        text=tick_labels,
        textfont=dict(size=tick_font_size),
        hoverinfo='none',
        showlegend=False
    ))
    fig.add_trace(go.Scatter3d(
        x=[0.0 + half_range*0.01]*len(tick_positions), y=[0.0 + half_range*0.01]*len(tick_positions), z=tick_positions,
        mode='text',
        text=tick_labels,
        textfont=dict(size=tick_font_size),
        hoverinfo='none',
        showlegend=False
    ))

    # Layout: remove background planes and default axis lines/ticks; we use custom axes/ticks (unchanged)
    scene = dict(
        xaxis=dict(title='X (AU)', showgrid=False, showbackground=False, showline=False, showticklabels=False, range=axis_range),
        yaxis=dict(title='Y (AU)', showgrid=False, showbackground=False, showline=False, showticklabels=False, range=axis_range),
        zaxis=dict(title='Z (AU)', showgrid=False, showbackground=False, showline=False, showticklabels=False, range=axis_range),
        aspectmode='cube',
        camera=dict(center=dict(x=0.0, y=0.0, z=0.0), eye=dict(x=1.25, y=1.25, z=1.25))
    )

    fig.update_layout(title='Trajectories with epoch-colored markers (same JD -> same color)', scene=scene, margin=dict(r=10, l=10, b=10, t=40))
    fig.show()


In [29]:
N_POINTS_FETCH = 3000

TILL_DATE = '2025-12-01'

df_v1 = fetch_vectors_horizons('Voyager 1', '1977-09-06', TILL_DATE, N_POINTS_FETCH)
df_v2 = fetch_vectors_horizons('Voyager 2', '1977-08-21', TILL_DATE, N_POINTS_FETCH)
df_p10 = fetch_vectors_horizons('Pioneer 10', '1972-03-04', TILL_DATE, N_POINTS_FETCH)
df_p11 = fetch_vectors_horizons('Pioneer 11', '1973-04-07', TILL_DATE, N_POINTS_FETCH)
df_nh = fetch_vectors_horizons('-98', '2006-01-20', TILL_DATE, N_POINTS_FETCH)

df_Earth = fetch_vectors_horizons('3', '1972-01-01', TILL_DATE, N_POINTS_FETCH)
#df_Mars = fetch_vectors_horizons('4', '1972-01-01', TILL_DATE, N_POINTS_FETCH)
df_Jupiter = fetch_vectors_horizons('5', '1972-01-01', TILL_DATE, N_POINTS_FETCH)
df_Saturn = fetch_vectors_horizons('6', '1972-01-01', TILL_DATE, N_POINTS_FETCH)
df_Uranus = fetch_vectors_horizons('7', '1972-01-01', TILL_DATE, N_POINTS_FETCH)
df_Neptune = fetch_vectors_horizons('8', '1972-01-01', TILL_DATE, N_POINTS_FETCH)

plot_3d_axes_at_origin(['Voyager 1', 'Voyager 2', 'Pioneer 10','Pioneer 11', 'New Horizon', 'Earth', 'Jupiter', 'Saturn', 'Uranus', 'Neptune'],
                       [df_v1, df_v2, df_p10, df_p11, df_nh, df_Earth, df_Jupiter, df_Saturn, df_Uranus, df_Neptune])


Using range request to Horizons with step = 6 day(s) to avoid long URL.
Using range request to Horizons with step = 6 day(s) to avoid long URL.
Using range request to Horizons with step = 7 day(s) to avoid long URL.
Using range request to Horizons with step = 6 day(s) to avoid long URL.
Using range request to Horizons with step = 2 day(s) to avoid long URL.
Using range request to Horizons with step = 7 day(s) to avoid long URL.
Using range request to Horizons with step = 7 day(s) to avoid long URL.
Using range request to Horizons with step = 7 day(s) to avoid long URL.
Using range request to Horizons with step = 2 day(s) to avoid long URL.
Using range request to Horizons with step = 7 day(s) to avoid long URL.
Using range request to Horizons with step = 7 day(s) to avoid long URL.
Using range request to Horizons with step = 7 day(s) to avoid long URL.
Using range request to Horizons with step = 7 day(s) to avoid long URL.
Using range request to Horizons with step = 7 day(s) to avoid lo